# STAT 344 GROUP PROJECT



In [31]:
# Run this cell before continuing.
library(dplyr)
library(ggplot2)
library(tibble)
library(infer)



In [2]:
student_df<-read.csv("https://raw.githubusercontent.com/Yuji03b/STAT-344-group-project/main/student_data.csv",header=T)

### 1. How to decide the stratified?
- create a table to show the varience between each strata, the larger the best, there are two reasonable strat "school" and "gender"

### 3. Our first parameter is : avergae final grade G3 

In [4]:
n <- 100             #(sample size
N <- nrow(student_df)  # population size

female_data <- subset(student_df,sex=="F")  # the female population 
male_data <- subset(student_df,sex=="M")  # the male population

female_N<-nrow(female_data)    # the female student population size
male_N<-nrow(male_data)      # the male student population size

In [5]:
sd(female_data$G3)
sd(male_data$G3)
### !!can make some comment on the difference in subpopulation standard deviation

[1] 4.622338

[1] 4.495297

### 3.1 Stratified sample :using proportional allocation N_k/N

In [6]:
#using proportional allocation N_k/N
set.seed(12345)
female_n <- round(n*(female_N/N))
male_n <- round(n*(male_N/N))
female_sample <- rep_sample_n(female_data,size=female_n,replace=F)
male_sample <- rep_sample_n(male_data,size=male_n,replace=F)

In [7]:
stratified_mean_grade <- mean(female_sample$G3)*(female_N/N)+mean(male_sample$G3)*(male_N/N)  # estimate of stratified mean


# standard error of stratified mean grade
female_FPC <- 1-female_n/female_N
male_FPC <- 1-male_n/male_N

se_stratified<-sqrt((female_N/N)^2*female_FPC*var(female_sample$G3)/female_n+
                   (male_N/N)^2*male_FPC*var(male_sample$G3)/male_n)


#95% confidence interval
stratified_CI<- c(stratified_mean_grade - 1.96* se_stratified,stratified_mean_grade + 1.96* se_stratified)

In [8]:
#check within variance roungly the same , we will use proportional allocation
(female_N/N)*female_FPC*var(female_sample$G3)/female_n
(male_N/N)*male_FPC*var(male_sample$G3)/male_n


[1] 0.1230289

[1] 0.1493146

### Stratified sample :using optical allocation N_k/N

In [29]:
c = 100/(male_N * sd(male_data$G3) + female_N * sd(female_data$G3) )
n_female_continous = c* female_N * sd(female_data$G3)
 n_male_continous = c* male_N * sd(male_data$G3)

n_female_continous
female_n

#optimal allocation 和 proportional allocation 给的sample size差不多，所以我们还是用proportional alloction. 

[1] 53.35242

[1] 46.64758

[1] 53

### Random select sample (SRS)

In [9]:
set.seed(12345)
SRS_sample <- sample_n(student_df,size=n,replace=F)

SRS_mean_grade <-mean(SRS_sample$G3) # estimate mean of SRS
se_SRS <- sqrt(var(SRS_sample$G3)/n) #standard error of SRS estimate

SRS_CI<- c(SRS_mean_grade - 1.96* se_SRS,SRS_mean_grade + 1.96* se_SRS)

In [10]:
average_G3_score <- c(SRS_mean_grade,stratified_mean_grade)
sampling_method <- c("SRS","stratified method")
standard_error <- c(se_SRS,se_stratified)


good_table <- data.frame("sampling method"=sampling_method,"average G3 score"=average_G3_score,"standard error"=standard_error,
                         "CI lower" = c(SRS_CI[1],stratified_CI[1]),"CI upper" = c(SRS_CI[2],stratified_CI[2]))
good_table

sampling.method,average.G3.score,standard.error,CI.lower,CI.upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SRS,9.97000,0.4600077,9.068385,10.87162
stratified method,11.09977,0.3680666,10.378356,11.82118



### 4. The second parmeter is : proportion of student who get 90% score in G3 (18 out of 20)
        !!!  second paramter is binary population

In [11]:
A_score_df <- subset(student_df,(G3<=20)&(G3>=16))

nrow(A_score_df)/N  # true proportion

A_score_df


[1] 0.1012658

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,⋯,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
9,GP,M,15,U,LE3,A,3,2,services,other,⋯,4,2,2,1,1,1,0,16,18,19
15,GP,M,15,U,GT3,A,2,2,other,other,⋯,4,5,2,1,1,3,0,14,16,16
23,GP,M,16,U,LE3,T,4,2,teacher,other,⋯,4,5,1,1,3,5,2,15,15,16
32,GP,M,15,U,GT3,T,4,4,services,services,⋯,4,3,1,1,1,5,0,17,16,17
33,GP,M,15,R,GT3,T,4,3,teacher,at_home,⋯,4,5,2,1,1,5,0,17,16,16
37,GP,M,15,U,LE3,T,4,3,teacher,services,⋯,5,4,3,1,1,4,2,15,16,18
43,GP,M,15,U,GT3,T,4,4,services,teacher,⋯,4,3,3,1,1,5,2,19,18,18
48,GP,M,16,U,GT3,T,4,3,health,services,⋯,4,2,2,1,1,2,4,19,19,20
60,GP,F,16,U,GT3,T,4,2,services,other,⋯,4,2,3,1,1,5,2,15,16,16


In [12]:
nrow(A_score_df)/N *100

[1] 10.12658

In [13]:
16/20

[1] 0.8

### SRS for A score

In [14]:
SRS_proportion_Ascore <- nrow(subset(SRS_sample,(G3<=20)&(G3>=16)))/n # 
se_SRS_Ascore <- sqrt(SRS_proportion_Ascore*(1-SRS_proportion_Ascore)/n) #standard error of SRS estimate

SRS_CI_Ascore<- c(SRS_proportion_Ascore - 1.96* se_SRS_Ascore,SRS_proportion_Ascore + 1.96* se_SRS_Ascore)


SRS_proportion_Ascore
se_SRS_Ascore

[1] 0.08

[1] 0.02712932

### Stratified sampling (proportional allocation) for A score

In [15]:
female_Ascore <- nrow(subset(female_sample,(G3<=20)&(G3>=16)))/female_n
male_Ascore <- nrow(subset(male_sample,(G3<=20)&(G3>=16)))/male_n

stratified_proportion_Ascore <- female_Ascore*(female_N/N)+male_Ascore*(male_N/N)

se_stratified_Ascore <-sqrt((female_N/N)^2*female_FPC*(female_Ascore*(1-female_Ascore))/female_n+
                   (male_N/N)^2*male_FPC*(male_Ascore*(1-male_Ascore))/male_n)
stratified_CI_Ascore<- c(stratified_proportion_Ascore - 1.96* se_stratified_Ascore,stratified_proportion_Ascore + 1.96* se_stratified_Ascore)


In [16]:
female_Ascore
male_Ascore

[1] 0.1132075

[1] 0.08510638

In [17]:
A_prop <- c(SRS_proportion_Ascore,stratified_proportion_Ascore)
standard_error_A <- c(se_SRS_Ascore,se_stratified_Ascore)
good_table_2 <- data.frame("sampling method"=sampling_method,"proportion of A score"=A_prop,"standard error"=standard_error_A,
                         "CI lower" = c(SRS_CI_Ascore[1],stratified_CI_Ascore[1]),"CI upper" = c(SRS_CI_Ascore[2],stratified_CI_Ascore[2]))
good_table_2

sampling.method,proportion.of.A.score,standard.error,CI.lower,CI.upper
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SRS,0.08000000,0.02712932,0.02682653,0.1331735
stratified method,0.09990396,0.02587237,0.04919412,0.1506138


In [18]:
#check within variance, we might check the the optimal alloction
(female_N/N)*female_FPC*(female_Ascore*(1-female_Ascore))/female_n
(male_N/N)*male_FPC*(male_Ascore*(1-male_Ascore))/male_n

[1] 0.0007432863

[1] 0.0005871726

### Stratified sampling (optimal allocation) for A score

In [21]:
#true p of A score 
# n_female = c* female_N * (female_data$G3*(1-female_data$G3))
# n_male = c* male_N * (male_data$G3*(1-male_data$G3))

true_p_female <- nrow(subset(female_data,(G3<=20)&(G3>=16)))/nrow(female_data)
true_p_male  <- nrow(subset(male_data,(G3<=20)&(G3>=16)))/nrow(male_data)

In [23]:
true_p_male
true_p_female+true_p_male

nrow(A_score_df)/N

[1] 0.1283422

[1] 0.2052653

[1] 0.1012658